## Unit 2:  Homework Option III -- Working With Large Files

This homework provides students with the option to get practice opening, sampling, and cleaning data when it has file sizes that might stretch your available RAM, or even exceed it.  

**What you'll turn in:**

You'll make a pull request with a folder (titled as your name) that contains the following:

 - for section I, a file called `titanic`, stored in a binary `feather` or `parquet` file format, that reduces the `titanic` dataset to its smallest possible memory amount without tampering any of its values.
 - for section II, simply turn in this notebook with the questions answered.
 - for section III, a script called `chunking.py` that, when run, will connect to an s3 bucket, and stream a file through memory, clean it, and output it to a `.csv` file, even though the entire file was never entirely loaded into RAM.

### Section I: Downcasting the Titanic Dataset

**What You'll Learn:** The basics of managing memory within files, and how to use advanced file formats such as the `feather` or `parquet`, to make it easier to maintain persistent data types when a file isn't loaded.

**What You'll Turn In:** A file called `titanic`, which contains the memory-reduced form of the file.

#### Downcasting

`Downcasting` is the task of reducing the memory footprint of different columns in your dataset so they take up less RAM when you load them in.  

Most software used for handling data makes use of your available RAM to process its tasks.  If the size of your file neatly fits into the available RAM that your computer has then this is fine.  If it's significantly larger (no laptop is going to have 1TB of RAM for example), then you won't be able to load in the file and work with it.

Pandas works this way, and therefore the amount of working RAM you have available to use is going to function as a limit for what file sizes you can work with.  

##### A Quick Intro to Data Types in Pandas & Numpy

Numbers come in different flavors in pandas and numpy.  At the simplest level you have integers (whole numbers) and floating point numbers (numbers with decimals).  

However, numbers use different sized containers to store their values.  They are as follows:

 - **64 bit:** Can store values as large as 2<sup>64</sup>, which is 18446744073709551616
 - **32 bit:** Can store values as large as 2<sup>32</sup>, which is 4294967296
 - **16 bit:** Good for values up to 2<sup>16</sup>, 65536
 - **8 bit:**  2<sup>8</sup>, or up to 264
 
**Integers** typically have the above range, while **floats** typically can only go down to 32 bits.

You can see the whole range of numeric data types here:  https://docs.scipy.org/doc/numpy/user/basics.types.html

The important detail here, is that if a number is encoded as being a 64 bit number, it will *always* use the same amount of memory to store it, even if the value itself is much smaller.  

So, if you have a column of 0's and 1's, an 8 bit encoding will work perfectly fine (since the values are less than 2<sup>8</sup>), and a 64 bit encoding will take up 8x as much memory as it needs to.

An important detail about how Pandas works is that *all numbers are automatically encoded as 64 bit numbers*.  This is good for making sure values aren't tampered with, but bad for optimizing memory with large files.

**Methods Used For Managing Memory In Pandas:**

 - `df.memory_usage()`, returns the memory usage, in bytes, of whatever is selected.
 - `pd.Series.astype()`, allows you to change the data type of 1 variable to another.
 - `df.info()`, returns the data type and memory usage of every column selected
 - `df.dtypes()/pd.Series.dtype()`, returns the data type of everything selected
 
Let's take a look at how these items work.  Run the following cells to get a quick demonstration of what they do.

In [122]:
# load in the titanic dataset here -- use a different url if need be to
# load it in
import pandas as pd
import numpy as np
df=pd.read_csv(r'C:\Users\escag\es_repo280720\Homework\Unit2\data\titanic.csv')

In [123]:
# check the info of your dataset -- notice the 64 bit numbers
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [124]:
# We can also check the memory usage of each column
df.memory_usage()

Index           128
PassengerId    7128
Survived       7128
Pclass         7128
Name           7128
Sex            7128
Age            7128
SibSp          7128
Parch          7128
Ticket         7128
Fare           7128
Cabin          7128
Embarked       7128
dtype: int64

In [125]:
# now let's see what happens when we adjust a columns data type
df['Survived'].astype(np.int8).memory_usage()

1019

As you can see, an 8 bit number takes up about 1/8 as much memory as a 64 bit number.  Permanently changing something's data type is simple.

In [126]:
# this will change the data type of a column to something else
df['Survived'] = df['Survived'].astype(np.int8)

In [127]:
# and now we can see its memory footprint is permanently smaller
df.memory_usage()

Index           128
PassengerId    7128
Survived        891
Pclass         7128
Name           7128
Sex            7128
Age            7128
SibSp          7128
Parch          7128
Ticket         7128
Fare           7128
Cabin          7128
Embarked       7128
dtype: int64

Notice however, that if you make a columns data type *smaller* than what it is, the original values will be tampered.  For example, the Passenger ID column has values as large as 891.....which is more than 2<sup>8</sup>.  Notice what happens when you make the change:

In [128]:
df.PassengerId.max()

891

In [129]:
# the values at the end of the series should be 889, 890, 891, etc
df['PassengerId'].astype(np.int8)

0        1
1        2
2        3
3        4
4        5
      ... 
886    119
887    120
888    121
889    122
890    123
Name: PassengerId, Length: 891, dtype: int8

In [130]:
df.PassengerId.astype(np.int32)

0        1
1        2
2        3
3        4
4        5
      ... 
886    887
887    888
888    889
889    890
890    891
Name: PassengerId, Length: 891, dtype: int32

So clearly, getting things 'just right' is important.  Notice also the difference between **signed** and **unsigned** data types.  If it's signed, that means they can accept negative values.  

So, a datatype of `np.uint8` can accept ranges of 0 - 255, whereas `np.int8` accepts values from -128 to 127.

##### Categorical Data in Pandas and Numpy

Text based data in Pandas and Numpy has two different varieties:

 - **np.object**: this is the default numpy way of treating and handling data.  Pretty close to a python string, and is used to store data that doesn't have any other characteristic (integer, float, bool, etc)
 - **category**: this is a special data type built specifically for Pandas, to handle text data that has a small number of repeating values.  Like the `sex` column in our dataset.  When appropriately used, it can drastically reduce the memory footprint of text based data.  You can read more about them here:  https://pandas.pydata.org/pandas-docs/stable/user_guide/categorical.html
 
See below for a demonstration.

In [131]:
# the Sex column currently uses the same memory as a 64 bit number
df['Sex'].memory_usage()

7256

In [132]:
# if you change it to a category, it uses about the same amount
# as an 8 bit number
df['Sex'].astype('category').memory_usage()

1115

Please note that using categories only has the desired effect when there are repeating values.  Make sure to check the memory footprint before making the change!

#### Section I Task:  Clean Up the Titanic Dataset

For the first part of your homework assignment, your task is to reduce the memory footprint of the titanic dataset as much as possible, while not tampering any values, and export it to a binary file format called `feather`(preferably), or `parquet` which will maintain information about all of its data types when it's loaded back in. 

You can read more about the feather file format here:  https://blog.rstudio.com/2016/03/29/feather/.

The file `titanic` should be turned in inside a folder with your name on it, and it will be inspected to make sure it was downcasted in the most appropriate manner.

You can use the `to_feather()`, and `read_feather()` methods to load and export your files.  Or the `to_parquet()` and `read_parquet()` methods, respectively.

**Note:** It is best to use feather files if you can.  They are the only file format that can maintain persistent information about the `category` data type after it's been saved.

However, you will likely need some additional libraries to get it working.  Often pandas does not work well with `feather` files right out of the box.  The typical library used to work with feather files is, appropriately called......feather.  To install the `feather` library, simply go to Anaconda Prompt/Terminal and type in `conda install feather-format`.

The library's homepage can be found here:  https://pypi.org/project/feather-format/

This library isn't always well supported, and there's a possibility that you might have an exceedingly difficult time getting it to work.  The purpose of this homework is not to have you go down rabbit holes getting an obscure library to work.  If after 30 minutes - 1 hour you still don't have the file installed, feel free to use the `parquet` file format, which can be a little easier to work with.

You can find more about it here:  https://arrow.apache.org/docs/python/parquet.html

In [133]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [134]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int8   
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(4), int8(1), object(5)
memory usage: 77.6+ KB


In [135]:
df.describe().loc[['min',  'max']]

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
min,1.0,0.0,1.0,0.42,0.0,0.0,0.0000
max,891.0,1.0,3.0,80.00,8.0,6.0,512.3292


In [136]:
#Re-define types for columns
df.PassengerId=df.PassengerId.astype(np.int16)
df.Survived=df.Survived.astype(np.int8)
df.Pclass=df.Pclass.astype(np.int8)
df.Age=df.Age.astype(np.float32)
df.SibSp=df.SibSp.astype(np.int8)
df.Parch=df.Parch.astype(np.int8)
df.Fare=df.Fare.astype(np.float32)
df.Sex=df['Sex'].astype('category')
df.Embarked=df.Embarked.astype('category')

In [137]:
#reduced-size Dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   PassengerId  891 non-null    int16   
 1   Survived     891 non-null    int8    
 2   Pclass       891 non-null    int8    
 3   Name         891 non-null    object  
 4   Sex          891 non-null    category
 5   Age          714 non-null    float32 
 6   SibSp        891 non-null    int8    
 7   Parch        891 non-null    int8    
 8   Ticket       891 non-null    object  
 9   Fare         891 non-null    float32 
 10  Cabin        204 non-null    object  
 11  Embarked     889 non-null    category
dtypes: category(2), float32(2), int16(1), int8(4), object(3)
memory usage: 35.1+ KB


In [138]:
import feather
df.to_feather('C:\\Users\\escag\\es_repo280720\\Homework\\Unit2\\ES_Homework_2\\titanic_new.ftr')

In [149]:
df_ftr=pd.read_feather('C:\\Users\\escag\\es_repo280720\\Homework\\Unit2\\ES_Homework_2\\titanic_new.ftr')

In [150]:
df_ftr.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.250000,None,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.283302,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.925000,None,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.099998,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.050000,None,S


In [151]:
df_ftr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   PassengerId  891 non-null    int16   
 1   Survived     891 non-null    int8    
 2   Pclass       891 non-null    int8    
 3   Name         891 non-null    object  
 4   Sex          891 non-null    category
 5   Age          714 non-null    float32 
 6   SibSp        891 non-null    int8    
 7   Parch        891 non-null    int8    
 8   Ticket       891 non-null    object  
 9   Fare         891 non-null    float32 
 10  Cabin        204 non-null    object  
 11  Embarked     889 non-null    category
dtypes: category(2), float32(2), int16(1), int8(4), object(3)
memory usage: 35.1+ KB


In [145]:
df0=pd.read_csv(r'C:\Users\escag\es_repo280720\Homework\Unit2\data\titanic.csv')

In [148]:
df0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [146]:
df_ftr.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204247
std,257.353842,0.486592,0.836071,14.526491,1.102743,0.806057,49.693409
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329224


In [147]:
df0.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


### Section II: Working With Larger File Sizes

Section I was your warmup to get the hang of how to reduce the memory of your data and get it loaded into a more advanced file format that can be reused for other projects.

This section will extend what you just did, but add in two additional wrinkles:

 - The file will be much larger -- approximately 2 million rows
 - You'll find out what types of data types you should be working with.....*without* reading in the entire file in the first place.

Imagine this hypothetical scenario:  there's a 10 gb file that you need to work on, and loading it into pandas is returning a `MemoryError`.  You're almost certain you could reduce it to something much smaller......except you can't even read it in to figure out what to do next.

These types of chicken & egg problems are fairly common, and a popular antidote to them is to sample in a portion of your dataframe.  There are two primary ways to do this if you're reading in `.csv` files:

 - `nrows`   : tells you how many rows to read in from the original file
 - `skiprows`: tells you which rows to *skip* from the original file
 
For both of these you can just manually read in x number of rows relatively easily, but doing so right from the beginning or end has some problems.  Mainly, many datasets don't have consistent values from beginning to end.  

For example, if you have a dataset with 10 years of sales info, it's very possible values being recorded are very different at varying time segments.  Often new columns are added to datasets in the middle of their collection, and all values for all times before that are simply `null`, so just reading in the first 5000 or 10000 might not give you a consistent picture of what to expect.

For this reason, it's good to randomly sample in dataframes before you want to read them in entirely.

To do this, you can mix `lambda` functions (remember those?) with the `skiprows` argument, which can accept a function as values.

Here's the basic idea:

In [27]:
import random

# random.random() will generate a random value between 0 and 1
# so this will return True 50% of the time
random.random() > .50

True

Using this same logic, we can pass this into a lambda function like so, to read in 30% of the titanic dataset.

In [29]:
# use read_csv with a lambda function
df1 = pd.read_csv('C:\\Users\\escag\\es_repo280720\\Homework\\Unit2\\data\\titanic.csv', skiprows=lambda x: x > 0 and random.random() > 0.3)
# and notice the size of the dataframe that was read in
df1.shape

(259, 12)

Here, inside the `lambda` function, `x` represents the index value of the row being read in.  `x > 0` is used assuming the first row are headers, and of course `random.random() > 0.3` will return `True` 70% of the time, hence the results that we get.

The idea is that you can read in a very small fraction of a very large file if you want to investigate its most important properties.  

Now, this begs the question.....how do you specify the data types you want a column to be before you read it in?  

There is a very useful argument in `read_csv` called `dtype` that accepts a dictionary, where you can list column labels as keys, and their corresponding data type as a value.

So for example, if we wanted to change the `Embarked` column and the `Survived` column to `category` and `np.int8`, we could do so in the following way.

In [30]:
# this dictionary contains the columns we want to change
dtypes = {
    # column label: new data type
    'Embarked': 'category',
    'Survived': np.int8
       }
# and now we'll re-read in the .csv file, and use the dtypes dict
df1 = pd.read_csv('C:\\Users\\escag\\es_repo280720\\Homework\\Unit2\\data\\titanic.csv', dtype=dtypes)

In [31]:
# and we can see that the data type of the columns are in fact changed
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   PassengerId  891 non-null    int64   
 1   Survived     891 non-null    int8    
 2   Pclass       891 non-null    int64   
 3   Name         891 non-null    object  
 4   Sex          891 non-null    object  
 5   Age          714 non-null    float64 
 6   SibSp        891 non-null    int64   
 7   Parch        891 non-null    int64   
 8   Ticket       891 non-null    object  
 9   Fare         891 non-null    float64 
 10  Cabin        204 non-null    object  
 11  Embarked     889 non-null    category
dtypes: category(1), float64(2), int64(4), int8(1), object(4)
memory usage: 71.6+ KB


#### Section II Task: Sample, Clean Up, And Read in the taxi.csv file

This portion of section II contains the task that you will be graded on.  You can simply answer the prompts inside this notebook and turn it in.  No additional files are necessary.  The file is located in an S3 bucket at this location:  `https://dat-data.s3.amazonaws.com/taxi.csv`

It records information about every taxi ride given by a particular company for approximately 1 year.

**Part I:** Randomly sample in 10% the taxi.csv file

In [34]:
# your answer here
taxisR=pd.read_csv(r'C:\Users\escag\es_repo280720\Homework\Unit2\data\taxi.csv', skiprows=lambda x: x > 0 and random.random() > 0.1)
taxis=pd.read_csv(r'C:\Users\escag\es_repo280720\Homework\Unit2\data\taxi.csv') #load the whole file

In [35]:
#Whole file
taxis.shape

(1710670, 8)

In [36]:
#Sampled file ( 10% of taxi.csv)
taxisR.shape

(170918, 8)

**Part II:** Go ahead and do the appropriate exploratory data analysis to figure out the most appropriate data type for each column.

In [37]:
# your answer here
taxisR.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170918 entries, 0 to 170917
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   TRIP_ID       170918 non-null  int64  
 1   CALL_TYPE     170918 non-null  object 
 2   ORIGIN_CALL   36293 non-null   float64
 3   ORIGIN_STAND  80915 non-null   float64
 4   TAXI_ID       170918 non-null  int64  
 5   TIMESTAMP     170918 non-null  int64  
 6   DAY_TYPE      170918 non-null  object 
 7   MISSING_DATA  170918 non-null  bool   
dtypes: bool(1), float64(2), int64(3), object(2)
memory usage: 9.3+ MB


In [38]:
taxisR.head()

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA
0,1372637254620000657,A,39233.0,NaN,20000657,1372637254,A,False
1,1372640555620000235,C,NaN,NaN,20000235,1372640555,A,False
2,1372639875620000009,C,NaN,NaN,20000009,1372639875,A,False
3,1372639973620000686,B,NaN,23.0,20000686,1372639973,A,False
4,1372639096620000231,C,NaN,NaN,20000231,1372639096,A,False


In [47]:
taxisR.describe().loc[['min', 'max']]

,TRIP_ID,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP
min,1.372637e+18,2001.0,1.0,20000001.0,1.372637e+09
max,1.404172e+18,63882.0,63.0,20000981.0,1.404172e+09


In [39]:
taxisR.TRIP_ID.memory_usage()

1367472

In [40]:
taxisR.TRIP_ID.astype('object').memory_usage()

1367472

In [100]:
taxisR.ORIGIN_CALL.memory_usage()

1366840

In [50]:
taxisR.ORIGIN_CALL.fillna(-1).astype(np.uint16).memory_usage()

341964

In [51]:
taxisR.ORIGIN_CALL.astype(np.float32).memory_usage()

683800

In [52]:
taxisR.ORIGIN_STAND.memory_usage()

1367472

In [53]:
taxisR.ORIGIN_STAND.fillna(-1).astype(np.int8).memory_usage()

171046

In [54]:
taxisR.TIMESTAMP.memory_usage()

1367472

In [55]:
taxisR.TIMESTAMP.astype(np.int32).memory_usage()

683800

In [56]:
query=(taxis.ORIGIN_CALL.isnull())
taxis[~query].ORIGIN_CALL.apply(float.is_integer).all()

True

In [57]:
query=(taxis.ORIGIN_STAND.isnull())
taxis[~query].ORIGIN_STAND.apply(float.is_integer).all()

True

**Part III**: Create a dictionary that contains the key/value pairs for each column that needs to be changed to a different data type, and then read in the file.

In [58]:
# your answer here
dtypes={
    'TRIP_ID'     : 'object',
    'ORIGIN_CALL' : np.float32,
    'ORIGIN_STAND': np.float32,
    'TAXI_ID'     : np.int32,
    'TIMESTAMP'   : np.uint32,
    'CALL_TYPE'   : 'category',
    'DAY_TYPE'    : 'category'
}

In [66]:
taxisR=pd.read_csv(r'C:\Users\escag\es_repo280720\Homework\Unit2\data\taxi.csv', skiprows=lambda x: x > 0 and random.random() > 0.1, dtype=dtypes )

In [67]:
taxisR.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170852 entries, 0 to 170851
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype   
---  ------        --------------   -----   
 0   TRIP_ID       170852 non-null  object  
 1   CALL_TYPE     170852 non-null  category
 2   ORIGIN_CALL   36268 non-null   float32 
 3   ORIGIN_STAND  80846 non-null   float32 
 4   TAXI_ID       170852 non-null  int32   
 5   TIMESTAMP     170852 non-null  uint32  
 6   DAY_TYPE      170852 non-null  category
 7   MISSING_DATA  170852 non-null  bool    
dtypes: bool(1), category(2), float32(2), int32(1), object(1), uint32(1)
memory usage: 4.4+ MB


In [68]:
# your answer here
taxis= pd.read_csv('C:\\Users\\escag\\es_repo280720\\Homework\\Unit2\\data\\taxi.csv', dtype=dtypes)

In [70]:
taxis.memory_usage()

Index                128
TRIP_ID         13685360
CALL_TYPE        1710774
ORIGIN_CALL      6842680
ORIGIN_STAND     6842680
TAXI_ID          6842680
TIMESTAMP        6842680
DAY_TYPE         1710678
MISSING_DATA     1710670
dtype: int64

**Part IV:** Confirm that each column has the appropriate data type

In [183]:
taxisR.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171829 entries, 0 to 171828
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype   
---  ------        --------------   -----   
 0   TRIP_ID       171829 non-null  object  
 1   CALL_TYPE     171829 non-null  category
 2   ORIGIN_CALL   36707 non-null   float32 
 3   ORIGIN_STAND  81095 non-null   float32 
 4   TAXI_ID       171829 non-null  int32   
 5   TIMESTAMP     171829 non-null  uint32  
 6   DAY_TYPE      171829 non-null  category
 7   MISSING_DATA  171829 non-null  bool    
dtypes: bool(1), category(2), float32(2), int32(1), object(1), uint32(1)
memory usage: 4.4+ MB


### Section III (Optional): File Streaming

File streaming is a way to get around memory limitations in pandas.  When you stream in a file, you spoon feed a portion of it into memory, and when you're finished, load in the next portion, and so on until there's nothing left. 

It's less convenient than regular file I/O, but it removes any sort of memory limit you might face when working with a file because it's never loaded at the same time. 

In this section of the homework assignment, you'll be tasked with performing basic cleaning operations on a file......without ever having to load it into memory.  This means what you accomplish in this section you could perform on a file of any size.

##### Basic Introduction to File Streams

When it's available, you can specify a file stream by using the `chunksize` argument, which specifies that you only read in so many lines at a time.

Notice how it works:

In [75]:
# when we read in the csv file, we'll set chunksize to 200
dfch=pd.read_csv(r'C:\Users\escag\es_repo280720\Homework\Unit2\data\taxi.csv', chunksize=200)
# notice that df is NOT a df.....it's a file stream
type(dfch)

pandas.io.parsers.TextFileReader

In [76]:
# and now if we want, we can 'chunk' in 250 rows at a time
dfch.get_chunk()

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA
0,1372636858620000589,C,NaN,NaN,20000589,1372636858,A,False
1,1372637303620000596,B,NaN,7.0,20000596,1372637303,A,False
2,1372636951620000320,C,NaN,NaN,20000320,1372636951,A,False
3,1372636854620000520,C,NaN,NaN,20000520,1372636854,A,False
4,1372637091620000337,C,NaN,NaN,20000337,1372637091,A,False
...,...,...,...,...,...,...,...,...
195,1372650908620000403,C,NaN,NaN,20000403,1372650908,A,False
196,1372649081620000171,B,NaN,25.0,20000171,1372649081,A,False
197,1372650494620000686,C,NaN,NaN,20000686,1372650494,A,False
198,1372649630620000011,C,NaN,NaN,20000011,1372649630,A,False


In [77]:
# and this would be the next 250 rows
dfch.get_chunk()

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA
200,1372649121620000486,B,NaN,25.0,20000486,1372649121,A,False
201,1372649085620000569,C,NaN,NaN,20000569,1372649085,A,False
202,1372649452620000436,C,NaN,NaN,20000436,1372649452,A,False
203,1372645400620000435,B,NaN,9.0,20000435,1372645400,A,False
204,1372647564620000454,A,38867.0,NaN,20000454,1372647564,A,False
...,...,...,...,...,...,...,...,...
395,1372653597620000477,C,NaN,NaN,20000477,1372653597,A,False
396,1372661440620000408,A,54140.0,NaN,20000408,1372661440,A,False
397,1372661219620000674,B,NaN,8.0,20000674,1372661219,A,False
398,1372660990620000901,A,47175.0,NaN,20000901,1372660990,A,False


With this basic syntax, you can lazily read in bits and pieces of a file at a time, without having to worry about being able to fit the entire file into RAM at once.  This is useful for doing some basic exploratory data analysis, and peek into a file, even if it's unreasonably large.

However, what if you wanted some way to go through the *entire* file, and make bulk changes to it?  

You can do that as well, by looping through the file stream. Here's a simple example, that prints off the total memory footprint for each chunk being read in:

In [79]:
import pandas as pd
import numpy as np
# this basically reads: for every chunk in the file stream
for chunk in pd.read_csv(r'C:\Users\escag\es_repo280720\Homework\Unit2\data\titanic.csv', chunksize=200):
    # do this to each chunk
    print(chunk.memory_usage().sum())

19328
19332
19332
19332
8868


Notice what we're doing here.  `chunk` is the portion of the file that you're reading in and operating on.  `pd.read_csv(.....)` in this case is the iterator that you're looping over, not the entire file itself.

What's a little difficult to get used to is that `chunk` isn't some stand alone file that you play with once you've loaded it in.  It's just a portion of a loop that you pass some commands to.

To finish this section of the homework, you'll need to write functions that allow you to stream in data, and perform specific operations on your data set, without having it entirely loaded into memory, or being able to see it.

These functions should be written in a file called `chunking.py`, and each of the following two functions should be able to be called from an IDE to observe the results.

**Function 1**

**Name:** `probe_df`

**Arguments:** 
 - `file_path`, str; required,  Location of file to read in.
 - `chunksize`, int, required, default value is 1000.  Size of the chunk to use when streaming in the file.
 
**Returns:** a dictionary encoded in the following way: 
 - each key is the name of a column within your dataset
 - the value for each key is another dictionary with the following key/value pairs:
   - `null values`: number of null values for that column
   - `dtype`: data type for that column
   - `avg_val`: average value for that column ( if numeric, otherwise don't include )
   
**Note:** The `chunksize` argument can be used with a variety of file types, but you can just assume that you'll be reading a csv file, and nothing more.

In [80]:
filepath='C:\\Users\\escag\\es_repo280720\\Homework\\Unit2\\data\\taxi.csv'

In [82]:
def probe_df(file_path,chunksize=1000):
    import pandas as pd
    import numpy as np
    from pandas.api.types import is_numeric_dtype
    cols=[]
    dict1={}
 
    for df in pd.read_csv(file_path, chunksize=chunksize):
        cols=df.columns
        for coln in cols:
            var1=df[coln].isnull().sum()
            var2=np.dtype(df[coln])
            if is_numeric_dtype(df[coln]):
                var3=df[coln].sum()
                var4=df[coln].notnull().sum()
                          
                
            else:
                var3=0
            if coln not in dict1:
                dict1[coln]={'null values': [var1],
                         'dtype' : [str(var2)],
                         'sums': [var3],
                         'nums' : [var4] }
            else:
                dict1[coln]['null values'].append(var1)
                dict1[coln]['sums'].append(var3)
                dict1[coln]['nums'].append(var4)
                
    for k in dict1:
        dict1[k]['null values']=sum(dict1[k]['null values'])
                    
        if dict1[k]['dtype'] != ['object'] and dict1[k]['dtype'] != ['bool']:
            dict1[k]['Avg_values']=sum(dict1[k]['sums'])/sum(dict1[k]['nums'])
 
        else: 
            dict1[k]['Avg_values']='N/A'
            
        dict1[k].pop('sums', None)
        dict1[k].pop('nums', None)

    return dict1
  
    

In [85]:
dict1=probe_df(filepath, chunksize=250)

C:\Users\escag\anaconda3\lib\site-packages\ipykernel_launcher.py:34: RuntimeWarning: overflow encountered in longlong_scalars


{'TRIP_ID': {'null values': 0,
  'dtype': ['int64'],
  'Avg_values': -2810564610072.737},
 'CALL_TYPE': {'null values': 0, 'dtype': ['object'], 'Avg_values': 'N/A'},
 'ORIGIN_CALL': {'null values': 1345900,
  'dtype': ['float64'],
  'Avg_values': 24490.363017792035},
 'ORIGIN_STAND': {'null values': 904091,
  'dtype': ['float64'],
  'Avg_values': 30.272381254657013},
 'TAXI_ID': {'null values': 0,
  'dtype': ['int64'],
  'Avg_values': 20000348.53118135},
 'TIMESTAMP': {'null values': 0,
  'dtype': ['int64'],
  'Avg_values': 1388622387.18325},
 'DAY_TYPE': {'null values': 0, 'dtype': ['object'], 'Avg_values': 'N/A'},
 'MISSING_DATA': {'null values': 0, 'dtype': ['bool'], 'Avg_values': 'N/A'}}

In [84]:
dict1

{'TRIP_ID': {'null values': 0,
  'dtype': ['int64'],
  'Avg_values': -2810564610072.737},
 'CALL_TYPE': {'null values': 0, 'dtype': ['object'], 'Avg_values': 'N/A'},
 'ORIGIN_CALL': {'null values': 1345900,
  'dtype': ['float64'],
  'Avg_values': 24490.363017792035},
 'ORIGIN_STAND': {'null values': 904091,
  'dtype': ['float64'],
  'Avg_values': 30.272381254657013},
 'TAXI_ID': {'null values': 0,
  'dtype': ['int64'],
  'Avg_values': 20000348.53118135},
 'TIMESTAMP': {'null values': 0,
  'dtype': ['int64'],
  'Avg_values': 1388622387.18325},
 'DAY_TYPE': {'null values': 0, 'dtype': ['object'], 'Avg_values': 'N/A'},
 'MISSING_DATA': {'null values': 0, 'dtype': ['bool'], 'Avg_values': 'N/A'}}

In [91]:
dict2={}
for k in dict1:
    if dict1[k]['null values'] != 0:
        if k not in dict2:
            dict2[k]=dict1[k]['Avg_values']

In [92]:
dict2

{'ORIGIN_CALL': 24490.363017792035, 'ORIGIN_STAND': 30.272381254657013}

**Function II**

**Name:** `write_df`

**Arguments:**

 - `file_path_read`   : str, required; location of file to read in.
 - `file_path_write`  : str, required; location of the file to write the new file out to
 - `chunksize`, int, required, default value is 1000.  Size of the chunk to use when streaming in the file.
 - `missing_vals`: dict, optional; accepts a dictionary as an argument with key/value pairs that list the column in the dataset(key) as well as the value to fill missing values with for that column(value).  The values in this dictionary will be used to fill missing values in the file at the location in `file_path_read`.
 
**Returns:**

This function will **not** return a value in the terminal.  What it **will** do is write a new file to the location specified in `file_path_write`.  

So, for example if you call `write_df('file/path/to/stream', 'file/path/to/write')`, a new file will appear in the location at `file/path/to/write` as the function is being called.

The big idea behind this file is that you'll be able to do data cleaning operations on a file you've never actually seen in your terminal before.

**Hint:** Pandas has a `to_csv()` method, with the option of appending lines to the end of it -- this is good to use when looping through the file stream.

**Note:** We will check to see that column headers are not added multiple times!

**To Test:** We will first call `probe_df` on the `taxi` dataset in its original location to get a dictionary with its missing values.  We will then use that value as a basis for the `missing_vals` argument in the `write_df` function, and use those to fill in its missing values.

If both of these functions work as intended, they will allow us to fill in the dataset's missing values without having looked at it, and you will receive full marks.

In [111]:
filepathread='C:\\Users\\escag\\es_repo280720\\Homework\\Unit2\\data\\taxi.csv'
filepathwrite='C:\\Users\\escag\\es_repo280720\\Homework\\Unit2\\data\\taxi_new3.csv'

In [112]:
def write_df(file_path_read, file_path_write, chunk_size=1000, missing_vals=None):
    import pandas as pd
    import numpy as np
    n=0
    for df in pd.read_csv(file_path_read, chunksize=chunk_size):
        if missing_vals != None:
            df=df.fillna(missing_vals)
        if n==0:
            df.to_csv(path_or_buf= file_path_write,mode='w', index=False)
        else:
            df.to_csv(path_or_buf= file_path_write,mode='a', header=False, index=False)
        n+=1
    
            
            

In [116]:
write_df(filepathread, filepathwrite, chunk_size=1000, missing_vals=dict2)
#write_df(filepathread, filepathwrite, chunk_size=1000)

In [117]:
taxis_new=pd.read_csv(filepathwrite)
#taxis=pd.read_csv(filepathread)

In [118]:
taxis_new.head(15)

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA
0,1372636858620000589,C,24490.363018,30.272381,20000589,1372636858,A,False
1,1372637303620000596,B,24490.363018,7.000000,20000596,1372637303,A,False
2,1372636951620000320,C,24490.363018,30.272381,20000320,1372636951,A,False
3,1372636854620000520,C,24490.363018,30.272381,20000520,1372636854,A,False
4,1372637091620000337,C,24490.363018,30.272381,20000337,1372637091,A,False
5,1372636965620000231,C,24490.363018,30.272381,20000231,1372636965,A,False
6,1372637210620000456,C,24490.363018,30.272381,20000456,1372637210,A,False
7,1372637299620000011,C,24490.363018,30.272381,20000011,1372637299,A,False
8,1372637274620000403,C,24490.363018,30.272381,20000403,1372637274,A,False
9,1372637905620000320,C,24490.363018,30.272381,20000320,1372637905,A,False


In [119]:
taxis.head(15)

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA
0,1372636858620000589,C,NaN,NaN,20000589,1372636858,A,False
1,1372637303620000596,B,NaN,7.0,20000596,1372637303,A,False
2,1372636951620000320,C,NaN,NaN,20000320,1372636951,A,False
3,1372636854620000520,C,NaN,NaN,20000520,1372636854,A,False
4,1372637091620000337,C,NaN,NaN,20000337,1372637091,A,False
5,1372636965620000231,C,NaN,NaN,20000231,1372636965,A,False
6,1372637210620000456,C,NaN,NaN,20000456,1372637210,A,False
7,1372637299620000011,C,NaN,NaN,20000011,1372637299,A,False
8,1372637274620000403,C,NaN,NaN,20000403,1372637274,A,False
9,1372637905620000320,C,NaN,NaN,20000320,1372637905,A,False


In [120]:
taxis.describe()

,TRIP_ID,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP
count,1.710670e+06,364770.000000,806579.000000,1.710670e+06,1.710670e+06
mean,1.388622e+18,24490.363018,30.272381,2.000035e+07,1.388622e+09
std,9.180944e+15,19624.290043,17.747840,2.112405e+02,9.180944e+06
min,1.372637e+18,2001.000000,1.000000,2.000000e+07,1.372637e+09
25%,1.380731e+18,6593.000000,15.000000,2.000017e+07,1.380731e+09
50%,1.388493e+18,18755.000000,27.000000,2.000034e+07,1.388493e+09
75%,1.396750e+18,40808.000000,49.000000,2.000052e+07,1.396750e+09
max,1.404173e+18,63884.000000,63.000000,2.000098e+07,1.404173e+09


In [121]:
taxis_new.describe()

,TRIP_ID,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP
count,1.710670e+06,1.710670e+06,1.710670e+06,1.710670e+06,1.710670e+06
mean,1.388622e+18,2.449036e+04,3.027238e+01,2.000035e+07,1.388622e+09
std,9.180944e+15,9.061914e+03,1.218669e+01,2.112405e+02,9.180944e+06
min,1.372637e+18,2.001000e+03,1.000000e+00,2.000000e+07,1.372637e+09
25%,1.380731e+18,2.449036e+04,2.800000e+01,2.000017e+07,1.380731e+09
50%,1.388493e+18,2.449036e+04,3.027238e+01,2.000034e+07,1.388493e+09
75%,1.396750e+18,2.449036e+04,3.027238e+01,2.000052e+07,1.396750e+09
max,1.404173e+18,6.388400e+04,6.300000e+01,2.000098e+07,1.404173e+09
